In [ ]:
from joblib import dump, load

In [ ]:
x_train_mol = load('x_train_mol.pkl')
x_test_mol = load('x_test_mol.pkl')
x_valid_mol = load('x_valid_mol.pkl')

In [ ]:
y_train = load('y_train.pkl')
y_test = load('y_test.pkl')
y_valid = load('y_valid.pkl')

In [ ]:
from rdkit import Chem
from rdkit.Chem import DataStructs
import numpy as np

In [ ]:
fp1 = [Chem.RDKFingerprint(m1) for m1 in x_train_mol]

In [ ]:
fp2 = [Chem.RDKFingerprint(m2) for m2 in x_test_mol]

In [ ]:
fp3 = [Chem.RDKFingerprint(m3) for m3 in x_valid_mol]

In [ ]:
def rdkit_numpy_convert(fp):
    output = []
    for f in fp:
        arr = np.zeros([1,])
        DataStructs.ConvertToNumpyArray(f, arr)
        output.append(arr)
    return np.asarray(output)

In [ ]:
x_train = rdkit_numpy_convert(fp1)

In [ ]:
x_test = rdkit_numpy_convert(fp2)

In [ ]:
x_valid = rdkit_numpy_convert(fp3)

In [ ]:
dump(x_valid, 'x_valid_rdkit.pkl', compress=3)

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [ ]:
seed = 42

In [ ]:
cv = StratifiedKFold(n_splits=10, random_state=seed)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scale = StandardScaler().fit(x_train)
x_train_scaled = scale.transform(x_train)
x_test_scaled = scale.transform(x_test)
x_valid_scaled = scale.transform(x_valid)

In [ ]:
from sklearn.svm import SVC

In [ ]:
param_grid = {'C': np.arange(10, 50, 1),
              'gamma': np.arange(0.00001, 0.001, 0.00001)}

In [ ]:
svm = GridSearchCV(SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=seed), param_grid=param_grid, cv=cv, n_jobs=10, verbose=1)

In [ ]:
svm.fit(x_train_scaled, y_train)

In [ ]:
svm.best_params_

In [ ]:
svm.best_score_

In [ ]:
svm.cv_results_['mean_test_score']

In [ ]:
svm.cv_results_['params']

In [ ]:
svm_2c8 = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=seed, C=36, gamma=0.0001)

In [ ]:
svm_2c8.fit(x_train_scaled, y_train)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import matthews_corrcoef, accuracy_score, roc_auc_score, make_scorer, recall_score

In [ ]:
matthews = make_scorer(matthews_corrcoef)
specificity = make_scorer(recall_score, pos_label=0)

In [ ]:
cross_val_score(svm_2c8, x_train_scaled, y_train, cv=cv, scoring='accuracy').mean()

In [ ]:
cross_val_score(svm_2c8, x_train_scaled, y_train, cv=cv, scoring='roc_auc').mean()

In [ ]:
cross_val_score(svm_2c8, x_train_scaled, y_train, cv=cv, scoring=matthews).mean()

In [ ]:
cross_val_score(svm_2c8, x_train_scaled, y_train, cv=cv, scoring='recall').mean()

In [ ]:
cross_val_score(svm_2c8, x_train_scaled, y_train, cv=cv, scoring=specificity).mean()

In [ ]:
y_pred = svm_2c8.predict(x_test_scaled)

In [ ]:
y_pred

In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
roc_auc_score(y_test, svm_2c8.decision_function(x_test_scaled))

In [ ]:
matthews_corrcoef(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred, pos_label=0)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
training_acc = []
test_acc = []

for i in np.arange(0.00005, 0.00016, 0.00001):
    svm_2c8 = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=seed, C=36, gamma=i)
    svm_2c8.fit(x_train_scaled, y_train)
    training_acc.append(cross_val_score(svm_2c8, x_train_scaled, y_train, cv=cv, scoring='accuracy').mean())
    y_pred = svm_2c8.predict(x_test_scaled)
    test_acc.append(accuracy_score(y_test, y_pred))
print(max(training_acc),(training_acc.index(max(training_acc)))+1) # index starts from 0
print(max(test_acc),(test_acc.index(max(test_acc)))+1)
plt.figure(figsize=(20, 10))
plt.plot(np.arange(0.00005, 0.00016, 0.00001), training_acc, label="training_acc")
plt.plot(np.arange(0.00005, 0.00016, 0.00001), test_acc, label="test_acc")
plt.xlabel("C")
plt.ylabel("Accuracy")
xticks = np.arange(0.00005, 0.00016, 0.00001)
plt.xticks(xticks)
plt.legend()

In [ ]:
training_acc = []
test_acc = []

for i in np.arange(1, 50, 1):
    svm_2c8 = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=seed, C=i, gamma=0.0001)
    svm_2c8.fit(x_train_scaled, y_train)
    training_acc.append(cross_val_score(svm_2c8, x_train_scaled, 
                                        y_train, cv=cv, scoring='accuracy').mean())
    y_pred = svm_2c8.predict(x_test_scaled)
    test_acc.append(accuracy_score(y_test, y_pred))
print(max(training_acc),(training_acc.index(max(training_acc)))+1) # index starts from 0
print(max(test_acc),(test_acc.index(max(test_acc)))+1)
plt.figure(figsize=(20, 10))
plt.plot(np.arange(1, 50, 1), training_acc, label="training_acc")
plt.plot(np.arange(1, 50, 1), test_acc, label="test_acc")
plt.xlabel("C")
plt.ylabel("Accuracy")
xticks = np.arange(1, 50, 1)
plt.xticks(xticks)
plt.legend()

In [ ]:
y_pred_valid = svm_2c8.predict(x_valid_scaled)

In [ ]:
accuracy_score(y_valid, y_pred_valid)

In [ ]:
roc_auc_score(y_valid, svm_2c8.decision_function(x_valid_scaled))

In [ ]:
matthews_corrcoef(y_valid, y_pred_valid)

In [ ]:
recall_score(y_valid, y_pred_valid)

In [ ]:
recall_score(y_valid, y_pred_valid, pos_label=0)

In [ ]:
dump(svm_2c8, '2c8_rdkit_svm.pkl', compress=3)